## POSTAL CODES OF CANADA (TORONTO)

## PART 1
To scrape data from the link https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, and obtain the table of postal codes in Canada and to transform the data into a pandas dataframe:

In [1]:
#import pandas and json to read the table from the wikipedia link
import pandas as pd
import numpy as np
import json
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


To assess only the cells that have an assigned borough. Ignoring cells with a borough that is 'Not assigned'

In [2]:
mask=df['Borough'].isin(['Not assigned'])
mas=df[~mask]
mas.head(10)
#ALTERNATIVELY
#toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
#toronto_df_dropna.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


To merge rows with similar postal codes, 

In [3]:
mas=mas.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ','.join(x))
mas.head(10)

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Not assigned
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge,Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens,Parkview Hill"


In [4]:
#first make values with 'Not assigned' to be represented as missing values   
mas['Neighbourhood'].replace({'Not assigned':np.nan}, inplace=True)    
mas.reset_index(inplace=True)
#to check for number of missing values
mas.isnull().sum()

Postcode         0
Borough          0
Neighbourhood    1
dtype: int64

In [10]:
# for Neighborhood="Not assigned", make the value the same as Borough
mas.reset_index(inplace=True)
for index, row in mas.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
mas.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [5]:
#fill in missing value one from Borough
mas['Neighbourhood'].fillna(mas['Borough'], inplace=True)
mas.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [6]:
mas.shape[0]

103

## PART 2

### Using the Geocoder package or the csv file to create dataframe with latitude and longitude

In [7]:
#Importing libraries to be used
#!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests 

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # import k-means from clustering stage

#!conda install -c conda-forge folium=0.5.0 --yes 

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [8]:
#Using geopy to get latitude and longitude of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [9]:
df=pd.read_csv('http://cocl.us/Geospatial_data')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df['Postal Code'].head()

0    M1B
1    M1C
2    M1E
3    M1G
4    M1H
Name: Postal Code, dtype: object

In [11]:
merged_df=mas.merge(df, left_on='Postcode', right_on='Postal Code')


In [12]:
merged_df.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494


In [24]:
if merged_df.loc[5, 'Postcode']==merged_df.loc[5, 'Postal Code']:
    print('yes')


yes


In [13]:
merged_df.drop('Postal Code', axis=1, inplace=True)

In [14]:
merged_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## Create map of Toronto

In [19]:
#Using geopy to get lat and lon of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Create a map of Toronto with Neighbourhood super imporsed on top

In [20]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

 ## Filter only boroughs that contain the word Toronto

In [21]:
# filter borough names that contain the word Toronto
borough_names = list(merged_df.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [22]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = merged_df[merged_df['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


# PART 3

### Map of Toronto with neighbourhoods containing the word 'Toronto' super impossed on it 

In [23]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

# Using the Foursquare API to explore the neighborhoods

In [25]:
# define Foursquare Credentials and Version
CLIENT_ID = '0TSNTEYQISCUSFIFLM4JKYP4ONUK5L01ZKNNXG4CBQZB4AHX' # your Foursquare ID
CLIENT_SECRET = 'KIZN1LIXOV4JG0BVNUT3QIKY33EQBZH54CSYGDN3DGIVHTXA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ', CLIENT_ID)
print('CLIENT_SECRET:', CLIENT_SECRET)

Your credentails:
CLIENT_ID:  0TSNTEYQISCUSFIFLM4JKYP4ONUK5L01ZKNNXG4CBQZB4AHX
CLIENT_SECRET: KIZN1LIXOV4JG0BVNUT3QIKY33EQBZH54CSYGDN3DGIVHTXA


In [26]:
toronto_df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [24]:
# Explore the first neighborhood in the dataframe
toronto_df_new.loc[0, 'Neighbourhood']

neighborhood_latitude = toronto_df_new.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df_new.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df_new.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront,Regent Park are 43.6542599, -79.3606359.


In [72]:
import requests
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Postcode'], toronto_df_new['Borough'], toronto_df_new['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [73]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1696, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


### Checking how many venues were returned for each postal code

In [74]:
# Number of postal codes
venues_df["PostalCode"].count()

1696

In [75]:
venue_post=venues_df.groupby(["PostalCode"]).count()
venue_post['VenueName']

PostalCode
M4E      4
M4K     43
M4L     20
M4M     40
M4N      3
M4P     11
M4R     20
M4S     36
M4T      2
M4V     15
M4W      4
M4X     46
M4Y     87
M5A     48
M5B    100
M5C    100
M5E     56
M5G     81
M5H    100
M5J    100
M5K    100
M5L    100
M5N      3
M5P      4
M5R     25
M5S     34
M5T    100
M5V     13
M5W     93
M5X    100
M6G     15
M6H     18
M6J     60
M6K     20
M6P     23
M6R     16
M6S     39
M7Y     17
Name: VenueName, dtype: int64

### Find out how many unique categories can be found from all returned venues

In [76]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 236 uniques categories.


In [78]:
venues_df['VenueCategory'].unique()[:100]

array(['Bakery', 'Coffee Shop', 'Gym / Fitness Center', 'Spa',
       'Breakfast Spot', 'Restaurant', 'Pub', 'Park', 'Historic Site',
       'Chocolate Shop', 'Farmers Market', 'Dessert Shop',
       'Mexican Restaurant', 'Performing Arts Venue', 'French Restaurant',
       'Theater', 'Café', 'Yoga Studio', 'Event Space',
       'Italian Restaurant', 'Shoe Store', 'Brewery', 'Art Gallery',
       'Asian Restaurant', 'Electronics Store', 'Beer Store', 'Bank',
       'Hotel', 'Health Food Store', 'Antique Shop', 'Clothing Store',
       'Burrito Place', 'Comic Shop', 'Pizza Place', 'Plaza',
       'Ramen Restaurant', 'Thai Restaurant', 'Diner', 'Tea Room',
       'Movie Theater', 'Taco Place', 'Music Venue', 'Shopping Mall',
       'Burger Joint', 'Vegetarian / Vegan Restaurant', 'Beer Bar',
       'Sandwich Place', 'American Restaurant', 'Steakhouse',
       'Concert Hall', 'Japanese Restaurant', 'Gastropub', 'Juice Bar',
       'Fast Food Restaurant', 'Middle Eastern Restaurant', 'Food

## Analyzing each area

In [80]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1696, 239)


,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Harbourfront,Regent Park",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Harbourfront,Regent Park",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group by neighborhood taking the frequency of occurance of each category

In [81]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head(8)

(38, 239)


,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,M4K,East Toronto,"The Danforth West,Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.023256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256
2,M4L,East Toronto,"The Beaches West,India Bazaar",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025000
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,M4P,Central Toronto,Davisville North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,M4R,Central Toronto,North Toronto West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000
7,M4S,Central Toronto,Davisville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


### We now create a dataframe desplaying the top 10 venues from each postalcode

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(38, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Neighborhood,Coffee Shop,Pub,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",Park,Sandwich Place,Italian Restaurant,Pet Store,Gym,Pub,Movie Theater,Burrito Place,Burger Joint,Brewery
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Park,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
5,M4P,Central Toronto,Davisville North,Breakfast Spot,Hotel,Clothing Store,Food & Drink Shop,Dance Studio,Sandwich Place,Burger Joint,Park,Gym,Gym / Fitness Center
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Italian Restaurant,Gym / Fitness Center,Chinese Restaurant,Dessert Shop,Rental Car Location,Diner
7,M4S,Central Toronto,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Restaurant,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Deli / Bodega,Diner
8,M4T,Central Toronto,"Moore Park,Summerhill East",Playground,Summer Camp,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Pub,Coffee Shop,Pizza Place,Spa,Bagel Shop,Fried Chicken Joint,Sports Bar,Supermarket,American Restaurant,Sushi Restaurant


## Cluster area

### Use Kmeans to cluster Toronto area into 5 clusters

In [103]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 1, 2, 2, 2, 0, 2])

In [104]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="Postcode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(38, 16)


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Brewery,Shoe Store
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Bar,Japanese Restaurant,Italian Restaurant,Thai Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Clothing Store,Cocktail Bar,Park,Cosmetics Shop,Italian Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Neighborhood,Coffee Shop,Pub,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Farmers Market,Café,Pub,Cheese Shop,Seafood Restaurant,Restaurant,Bakery,Italian Restaurant


In [105]:

# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(38, 16)


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259,0,Bakery,Pharmacy,Discount Store,Supermarket,Gym / Fitness Center,Pool,Music Venue,Middle Eastern Restaurant,Café,Brewery
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Farmers Market,Café,Pub,Cheese Shop,Seafood Restaurant,Restaurant,Bakery,Italian Restaurant
10,M6J,West Toronto,"Little Portugal,Trinity",43.647927,-79.419750,1,Bar,Coffee Shop,Asian Restaurant,Restaurant,Vietnamese Restaurant,Café,Bakery,Pizza Place,Cocktail Bar,Men's Store
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Brewery,Shoe Store
22,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Italian Restaurant,Gym / Fitness Center,Chinese Restaurant,Dessert Shop,Rental Car Location,Diner
23,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,2,Coffee Shop,Sandwich Place,Café,Pizza Place,BBQ Joint,Indian Restaurant,Jewish Restaurant,Liquor Store,Cosmetics Shop,Park
24,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325,2,Gift Shop,Breakfast Spot,Bookstore,Bar,Eastern European Restaurant,Restaurant,Italian Restaurant,Dog Run,Bank,Coffee Shop
25,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Dessert Shop,Sandwich Place,Pizza Place,Restaurant,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Deli / Bodega,Diner
26,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049,2,Café,Bar,Japanese Restaurant,Bookstore,Restaurant,Bakery,Coffee Shop,Pub,Beer Bar,Beer Store
27,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450,2,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Electronics Store,Italian Restaurant,Fish & Chips Shop,Bar,Food,Fish Market


### Finally to visualize the resulting clusters

In [106]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Borough'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining clusters
## Clusters

In [107]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] ==0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))                                                
                                                                                ]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,West Toronto,0,Bakery,Pharmacy,Discount Store,Supermarket,Gym / Fitness Center,Pool,Music Venue,Middle Eastern Restaurant,Café,Brewery


In [108]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] ==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))                                                
                                                                                ]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Farmers Market,Café,Pub,Cheese Shop,Seafood Restaurant,Restaurant,Bakery,Italian Restaurant
10,West Toronto,1,Bar,Coffee Shop,Asian Restaurant,Restaurant,Vietnamese Restaurant,Café,Bakery,Pizza Place,Cocktail Bar,Men's Store


In [109]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] ==2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))                                                
                                                                                ]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Brewery,Shoe Store
22,Central Toronto,2,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Italian Restaurant,Gym / Fitness Center,Chinese Restaurant,Dessert Shop,Rental Car Location,Diner
23,Central Toronto,2,Coffee Shop,Sandwich Place,Café,Pizza Place,BBQ Joint,Indian Restaurant,Jewish Restaurant,Liquor Store,Cosmetics Shop,Park
24,West Toronto,2,Gift Shop,Breakfast Spot,Bookstore,Bar,Eastern European Restaurant,Restaurant,Italian Restaurant,Dog Run,Bank,Coffee Shop
25,Central Toronto,2,Dessert Shop,Sandwich Place,Pizza Place,Restaurant,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Deli / Bodega,Diner
26,Downtown Toronto,2,Café,Bar,Japanese Restaurant,Bookstore,Restaurant,Bakery,Coffee Shop,Pub,Beer Bar,Beer Store
27,West Toronto,2,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Electronics Store,Italian Restaurant,Fish & Chips Shop,Bar,Food,Fish Market
28,Central Toronto,2,Playground,Summer Camp,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
29,Downtown Toronto,2,Café,Bar,Vegetarian / Vegan Restaurant,Chinese Restaurant,Vietnamese Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Caribbean Restaurant,Dessert Shop
30,Central Toronto,2,Pub,Coffee Shop,Pizza Place,Spa,Bagel Shop,Fried Chicken Joint,Sports Bar,Supermarket,American Restaurant,Sushi Restaurant


## Summary

Observing the 3 clusters we can see that cluster 1 is mainly made up of sotres such as the barkery, supermarket phamercy while cluster 2 is a mixture restaurants and stores which also mirrors cluster 3